In [1]:
import tensorflow as tf
from tensorflow import keras
import os
# Chay bang CPU thay vi GPU
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

class buildmodel():
    def conv_block(self,inputs , filters, block_num,conv_num=2):
        x = keras.layers.Conv2D(filters,3, activation = 'relu', padding = 'same',name = 'conv_%d_1'%block_num)(inputs)
        x = keras.layers.Conv2D(filters,3, activation = 'relu', padding = 'same',name = 'conv_%d_2'%block_num)(x)
        if conv_num==3:
            x = keras.layers.Conv2D(filters,3, activation = 'relu', padding = 'same',name = 'conv_%d_3'%block_num)(x)
        x = keras.layers.MaxPooling2D(name = 'max_pooling_%d'%block_num)(x)
        return x

    def vgg16(self,num_outputs=1000):
        inputs = keras.layers.Input(shape=(224,224,3),name = 'input')
        x = self.conv_block(inputs,64,1)
        x = self.conv_block(x,128,2)
        x = self.conv_block(x,256,3,3)
        x = self.conv_block(x,512,4,3)
        x = self.conv_block(x,512,5,3)
        x = keras.layers.Flatten()(x)
        x = keras.layers.Dense(4096,activation = 'relu',name = 'dense_1')(x)
        x = keras.layers.Dense(4096,activation = 'relu',name = 'dense_2')(x)
        outputs = keras.layers.Dense(num_outputs,activation = 'softmax',name = 'outputs')(x)
        model = keras.Model(inputs = inputs, outputs = outputs, name = 'vgg16')
        return model

    def residual_block(self,inputs, filter,block_num,reduce=False):
        res = inputs
        x = keras.layers.Conv2D(filter,(3, 3), strides = 2 if reduce else 1,activation = 'relu', padding = 'same',name = f'conv_{block_num}_1')(res)
        x = keras.layers.Conv2D(filter,(3, 3), activation = 'relu', padding = 'same',name = f'conv_{block_num}_2')(x)
        if reduce == True:
            res = keras.layers.Conv2D(filter,(1, 1), strides = 2, activation = 'relu', padding = 'same',name = f'conv_skip_{block_num-1}')(res)
        res = keras.layers.Add(name = f'end_res_bock_{2*block_num-1}')([x,res])
        x = keras.layers.Conv2D(filter,(3, 3), activation = 'relu', padding = 'same',name = f'conv_{block_num}_3')(res)
        x = keras.layers.Conv2D(filter,(3, 3), activation = 'relu', padding = 'same',name = f'conv_{block_num}_4')(x)
        res = keras.layers.Add(name = f'end_res_bock_{2*block_num}')([x,res])
        return res
        
    def resnet18(self,num_outputs=1000):
        # Inputs
        inputs = keras.layers.Input(shape=(224,224,3),name = 'input')
        x = keras.layers.Conv2D(64,(7, 7),strides = 2, activation = 'relu', padding = 'same',name = 'conv2D')(inputs)
        x = keras.layers.MaxPooling2D(pool_size = (3,3),strides = 2 ,padding = 'same',name = 'max_pool_1')(x)
        # Block 1
        x = self.residual_block(x,64,1,False)
        # Block 2
        x = self.residual_block(x,128,2,True)
        # Block 3
        x = self.residual_block(x,256,3,True)
        # Block 4
        x = self.residual_block(x,512,4,True)
        # Ouputs
        x = keras.layers.GlobalAveragePooling2D(name = 'global_pooling')(x)
        if num_outputs==1:
            outputs = keras.layers.Dense(num_outputs,activation = 'sigmoid',name = 'outputs_sigmoid')(x)
        else:
            outputs = keras.layers.Dense(num_outputs,activation = 'softmax',name = 'outputs_softmax')(x)
        model = keras.Model(inputs = inputs, outputs = outputs, name = 'resnet18')
        return model
    def __init__(self,model_name = 'resnet18',num_outputs=1):
        if model_name == 'resnet18':
            self.model = self.resnet18(num_outputs = num_outputs)
        if model_name == 'vgg16':
            self.model = self.vgg16(num_outputs=num_outputs)
        self.opt = keras.optimizers.Adam()
        self.loss_fn = keras.losses.BinaryCrossentropy()
        self.metric = keras.metrics.BinaryAccuracy()
        self.model.compile(optimizer=self.opt, loss= self.loss_fn)
    @tf.function
    def train_step(self,x, y):
        with tf.GradientTape() as tape:
            logits = self.model(x, training=True)
            loss_value = self.loss_fn(y, logits)
        grads = tape.gradient(loss_value, self.model.trainable_weights)
        self.metric.update_state(y,logits)
        return grads        
    @tf.function
    def update_grads(self, grads):
        self.opt.apply_gradients(zip(grads, self.model.trainable_weights))

2023-12-20 17:17:54.927344: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-20 17:17:54.927438: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-20 17:17:54.967837: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-20 17:17:55.052648: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-20 17:17:55.899838: W tensorflow/compiler/tf2

In [2]:
model = buildmodel().model

2023-12-20 17:17:56.991043: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-12-20 17:17:56.991070: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:129] retrieving CUDA diagnostic information for host: AILAB-PC
2023-12-20 17:17:56.991076: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:136] hostname: AILAB-PC
2023-12-20 17:17:56.991125: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:159] libcuda reported version is: 535.129.3
2023-12-20 17:17:56.991144: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:163] kernel reported version is: 535.129.3
2023-12-20 17:17:56.991149: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:241] kernel version seems to match DSO: 535.129.3


In [3]:
model.input_shape

(None, 224, 224, 3)

In [3]:
img_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
train_gen = img_gen.flow_from_directory('./../../../Sample Model/data',target_size=(224, 224),class_mode="binary")

Found 24959 images belonging to 2 classes.


In [14]:
import numpy

In [28]:
a = train_gen.next()

In [31]:
len(a[1])

60

In [5]:
import time

epochs = 1
for epoch in range(epochs):
    model = buildmodel()
    print("\nStart of epoch %d" % (epoch,))
    start_time = time.time()

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_gen):
        print(step)
        # grads = model.train_step(x_batch_train, y_batch_train)
        # model.update_grads(grads)
        # print(model.metric.result())

    # # Display metrics at the end of each epoch.
    # train_acc = train_acc_metric.result()
    # print("Training acc over epoch: %.4f" % (float(train_acc),))

    # # Reset training metrics at the end of each epoch
    # train_acc_metric.reset_states()

    # print("Time taken: %.2fs" % (time.time() - start_time))


Start of epoch 0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226


KeyboardInterrupt: 